## Utils

### Descriptions of knowledge profiles and mistakes

In [ ]:
knowledge_profiles = {
    '1111': "This knowledge profile represents student answers where the student correctly states what should be proven (rubric 1), makes the correct assumption (rubric 2), follows correct proof reasoning (rubric 3), and correctly concludes the proof (rubric 4). However, minor inconsistencies such as variations in notation or redundant steps may still be present.",
    '0000': "This knowledge profile represents student answers where the student fails to correctly state what should be proven (rubric 1), does not make the correct assumption (rubric 2), lacks correct proof reasoning (rubric 3), and does not conclude the proof correctly (rubric 4).",
    '1110': "This knowledge profile represents student answers where the student correctly states what should be proven (rubric 1), makes the correct assumption (rubric 2), and follows correct proof reasoning (rubric 3), but fails to correctly conclude the proof (rubric 4).",
    '0111': "This knowledge profile represents student answers where the student fails to correctly state what should be proven (rubric 1) but makes the correct assumption (rubric 2), follows correct proof reasoning (rubric 3), and correctly concludes the proof (rubric 4).",
    '0110': "This knowledge profile represents student answers where the student fails to correctly state what should be proven (rubric 1) but makes the correct assumption (rubric 2) and follows correct proof reasoning (rubric 3), but fails to correctly conclude the proof (rubric 4).",
    '1100': "This knowledge profile represents student answers where the student correctly states what should be proven (rubric 1) and makes the correct assumption (rubric 2) but does not follow correct proof reasoning (rubric 3) and fails to conclude the proof correctly (rubric 4).",
    '1000': "This knowledge profile represents student answers where the student correctly states what should be proven (rubric 1) but does not make the correct assumption (rubric 2), lacks correct proof reasoning (rubric 3), and does not conclude the proof correctly (rubric 4).",
    '0100': "This knowledge profile represents student answers where the student fails to correctly state what should be proven (rubric 1) but makes the correct assumption (rubric 2), while lacking correct proof reasoning (rubric 3) and not concluding the proof correctly (rubric 4)."
    }

### Prompt structure

Static parts of the prompt

In [ ]:
SYSTEM_ROLE = """
You are a text generator that generates realistic student responses for a proof by contraposition question. Your role is to simulate student answers based on provided knowledge profiles, which will be explained in more detail along with examples. Avoid including any evaluative language or comments that could bias the responses, as the generated data is used to train classifiers. Maintain a neutral, student-like tone and focus on coherence and clarity in your responses.
"""

In [ ]:
TASK_DESCRIPTION = """
TASK DESCRIPTION:
Students are expected to answer this task:
"Let x be a real number. Prove that if x^2 is irrational, then x is irrational using a proof by contraposition.

Reminders:
- A real number x is irrational if it cannot be expressed as the quotient of two integers - that is, p/q where p and q (both integers) are not zero.
- A real number x is rational if it can be expressed as the quotient of two integers - that is, p/q where p and q (both integers) are not zero."
"""

In [ ]:
SOLUTION = """
SOLUTION:
As a reference, here is the correct solution for this problem:
A proof by contraposition of an implication consists in showing that if x is rational, then x^2 is rational.
1. x is rational [Assumption]
2. x = a/b, where a and b ≠ 0 are integers with no common factors. [Definition of a rational number]
3. Therefore, x^2 = a^2/b^2. [Squaring]
4. Hence, x^2 is rational. [Definition of a rational number]
5. By contraposition, if x^2 is irrational, then x is irrational.
"""

In [ ]:
GRADING_RUBRICS = """
GRADING RUBRICS:
Students are graded using 4 binary rubrics:
1. Statement of what should be proven: A proof by contraposition of an implication consists in showing that if x is rational, then x^2 is rational.
2. Correct assumption: x is rational
3. Correct proof reasoning
4. Proof conclusion: By contraposition, if x^2 is irrational, then x is irrational.

It is acceptable if students do not write the exact same answer as the solution. If the response satisfies the rubrics more or less, the student receives the point.
"""

In [ ]:
KNOWLEDGE_PROFILES_EXPLANATION = """
KNOWLEDGE PROFILES:
We identify a student's knowledge profile based on their score in these 4 rubrics. For example:
- A student with all rubrics incorrect will have the profile 0-0-0-0.
- A student who has rubric 1 and 2 correct but 3 and 4 incorrect will have the profile 1-1-0-0.
"""

In [ ]:
OUTPUT_FORMAT = """
DESIRED OUTPUT FORMAT:
- Separate each generated student answer with the character "$".
- Do not include any comments or evaluative language since the data is used to train a classifier.
- Words like "correct", "incorrect", "right", "wrong", "correctly", and "incorrectly" are not allowed.
- When you generate an answer of a student that has mistakes, you have to act as a student who does not know the correct answer, and does not know that he is answering wrong.
- You have to generate an answer that is coherent with the knowledge profile of the student.
- At the end of each answer, include a grade formatted between two "&" characters, such as &1111& for a student scoring 1 in all rubrics or &0000& for a student scoring 0 in all. Please evaluate each generated answer based on the rubrics provided, grading it independently of the target knowledge profile specified in this prompt. Do not assume the answer matches the target profile; instead, assess it as if you were grading any student's answer without prior knowledge of the intended profile.
- Use the following syntax for formulas:
  - To put a to the power of b: (a POWER b) or a^b. Vary the syntax between them.
  - To divide a by b, use a/b.
- Structure your proof line by line, as shown in the example:
"Show that from p ∧ (p -> q) we can conclude q:
1) p ∧ (p -> q)
2) p  
3) (p -> q) 
4) q"
"""

### OpenAI Setup

In [ ]:
key = 'API key'

In [ ]:
from openai import OpenAI

OpenAI.api_key = key
client = OpenAI(api_key=key)

model="gpt-4o"
temperature=1

## Helper Functions

In [ ]:
import random
random.seed(42)
def get_answers_for_mistake_and_profile(df, knowledge_profile, profile_column='Knowledge Profile', answer_column='CONTRAPOSITION task'):

    filtered_df = df[df[profile_column] == knowledge_profile]
    answers = filtered_df[answer_column].tolist()

    random.shuffle(answers)
    
    return answers

In [ ]:
def construct_prompt(target_knowledge_profile, num_answers, example_answers):

    # define generic parts of the prompt
    target_knowledge_profile = f"""
        TARGET KNOWLEDGE PROFILE AND MISTAKE:
        I want you to generate answers for {num_answers} students with the knowledge profile {target_knowledge_profile}. {knowledge_profiles[target_knowledge_profile]} 
        You will generate student answers with this profile.
        Please ensure that each answer demonstrates the specified knowledge profile characteristics. Do not just generate an answer with the mistake and a correct proof otherwise; the answer should reflect the knowledge profile.
        The generated answers have to be different and diverse, do not just paraphrase the given examples.
        """

    examples = f"""
        EXAMPLES:
        Here are a few real answers from students who have this knowledge profile:
        {example_answers}
        """

    messages = [{"role": "system", "content": SYSTEM_ROLE},
                {"role": "user", "content": TASK_DESCRIPTION},
                {"role": "user", "content": SOLUTION},
                {"role": "user", "content": GRADING_RUBRICS},
                {"role": "user", "content": KNOWLEDGE_PROFILES_EXPLANATION},
                {"role": "user", "content": target_knowledge_profile},
                {"role": "user", "content": examples},
                {"role": "user", "content": OUTPUT_FORMAT}]
    
    return messages

In [ ]:
import json
from datetime import datetime
import os
def save_completion_with_metadata(completion, prompt_messages, knowledge_profile, n, num_answers, temperature, save_dir):
    # Add extra metadata to the completion object
    completion["metadata"] = {
        "prompt": prompt_messages,
        "knowledge_profile": knowledge_profile,
        "n_instances": n,
        "num_answers": num_answers,
        "temperature": temperature
    }
    
    # Generate filename with knowledge profile, date, and time
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{knowledge_profile}_{timestamp}.json"
    save_path = os.path.join(save_dir, knowledge_profile, filename)
    
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    with open(save_path, 'w') as json_file:
        json.dump(completion, json_file, indent=4)

    return save_path

In [ ]:
import json
import os
import re

def process_json_to_txt(input_json_path, output_dir):
    with open(input_json_path, 'r') as file:
        data = json.load(file)

    # Extract metadata from the filename
    json_filename = os.path.basename(input_json_path)
    base_filename = os.path.splitext(json_filename)[0]

    for choice_index, choice in enumerate(data.get("choices", [])):
        content = choice.get("message", {}).get("content", "")

        # Split content by '$' to separate each generated instance
        instances = content.split('$')
        
        for idx, instance in enumerate(instances):
            if instance.strip():  
                # Remove the grade formatted between '&' characters
                instance_cleaned = re.sub(r'&.*?&', '', instance).strip()

                knowledge_profile = data.get("metadata", {}).get("knowledge_profile", "unknown_profile")
                instance_dir = os.path.join(output_dir, knowledge_profile)
                os.makedirs(instance_dir, exist_ok=True)

                output_filename = f"{base_filename}_{choice_index + 1}_{idx + 1}.txt"
                output_path = os.path.join(instance_dir, output_filename)

                with open(output_path, 'w', encoding='utf-8') as txt_file:
                    txt_file.write(instance_cleaned)


## Pipeline

Read and identify knowledge profiles in the dataset.

In [ ]:
dataset_path = 'path/to/original/data'

In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_excel(dataset_path, sheet_name='contraposition')

# rubrics of the original dataset
rubrics = [
    'Statement of what should be proven: A proof by contraposition of an implication consists in showing that if x rational, then x^2 is rational. ',
    'Correct assumption: x is rational [Assumption] ',
    'Correct proof reasoning',
    'Proof conclusion: By contraposition, if x^2 is irrational, then x is irrational.'
]

df['Knowledge Profile'] = df[rubrics].astype(str).agg(''.join, axis=1)

Define prompt parameters

In [ ]:
profile = '0111'
num_answers = 2
num_examples = 5
examples_list = get_answers_for_mistake_and_profile(df, profile)
examples_count = min(num_examples, len(examples_list))
example_answers = "\n".join([f"- {answer}" for answer in examples_list[:examples_count]])

print("Example answers: ", example_answers, ' \n')
# construct the prompt 
prompt = construct_prompt(profile, num_answers, example_answers)
print("Prompt: ", prompt, '\n')

n=30
save_dir = f'raw_responses/temp_{temperature}'
output_dir = f'processed_responses/temp_{temperature}'

In [ ]:
completion = client.chat.completions.create(
    model=model,
    messages=prompt,
    temperature=temperature,
    n=n
)
path = save_completion_with_metadata(completion.to_dict(), prompt, profile, n, num_answers, temperature, save_dir=save_dir) 
process_json_to_txt(path, output_dir)